In [1]:
import torch
import torchvision.transforms.functional as F
from src import model, dataset, augmentations

In [2]:
class Resize(augmentations.T.Resize):
    def forward(self, image, mask):
        image = F.resize(image, self.size, self.interpolation)
        mask = F.resize(mask, self.size, self.interpolation)
        return (image,), (mask,)

In [3]:
dataset = dataset.CracksDataset('valid')
dataset.transform = augmentations.Compose([
    Resize(224),
    augmentations.ToTensor(),
])
loader = dataset.get_loader(batch_size=10, num_workers=0)
batch = next(iter(loader))

In [4]:
net = model.UNet().eval()
state_dict_segmentation = torch.load('logs/segmentation/checkpoints/best.pth', map_location='cpu')['model_state_dict']
net.load_state_dict(state_dict_segmentation, strict=False)
# state_dict_classification = torch.load('logs/classification/checkpoints/best.pth', map_location='cpu')['model_state_dict']
# net.load_state_dict(state_dict_classification, strict=False)

<All keys matched successfully>

In [5]:
targets = batch.pop('masks')
inputs = batch.pop('images')
outputs = net(inputs)

In [ ]:
save_fmt = 'tests/{}.png'
for index, (inp, tar, out) in enumerate(zip(inputs, targets, outputs)):
    F.to_pil_image(inp).save(save_fmt.format(f'input_{index}'))
    F.to_pil_image(tar).save(save_fmt.format(f'target_{index}'))
    F.to_pil_image(out).save(save_fmt.format(f'predict_{index}'))
    # print(inp.shape, tar.shape, out.shape)

In [ ]:
import torch
from src import model
net = model.UNetClassifier()
inputs1 = torch.rand(3, 224, 224)
inputs2 = torch.randn(3, 224, 224)
inputs = torch.stack((inputs1, inputs2))
masks, classes = net(inputs)

In [ ]:
masks[1].sum()

In [ ]:
torch.where(classes > 0.5)

In [ ]:
idx1

In [ ]:
import torch
filepath = 'logs/segmentation/checkpoints/last.pth'
state_dict = torch.load(filepath)
model_state_dict = state_dict['model_state_dict'].copy()
print(model_state_dict.keys())

In [ ]:
new_model_state_dict = {}
for key, value in model_state_dict.items():
    new_model_state_dict[key.replace('VGGEncoder', 'encoder')] = value

In [ ]:
state_dict['model_state_dict'] = new_model_state_dict

In [ ]:
torch.save(state_dict, filepath)